In [26]:
#  import the libraries needed

# Data handling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

# Data preprocessing
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE

# Machine learning (Deep Learning)
import tensorflow as tf
from tensorflow.keras.layers import InputLayer, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Machine learning (Traditional Models)
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb

# Model selection
from sklearn.model_selection import RandomizedSearchCV

# Model evaluation
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, classification_report

# Global settings
%matplotlib inline
warnings.filterwarnings(action='ignore', category=FutureWarning)
pd.set_option('display.max_row', 20)

In [27]:
# import the data to be used
ecom_cus_df = pd.read_excel("/content/drive/MyDrive/E Commerce Dataset.xlsx", sheet_name="E Comm")

In [28]:
# get a sample of the data to know how the data looks like
ecom_cus_df.sample(20)

,CustomerID,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
3442,53443,1,1.0,Mobile Phone,1,23.0,Debit Card,Female,3.0,2,Mobile Phone,5,Single,3,0,17.0,16.0,16.0,15.0,152.43
2906,52907,1,1.0,Computer,3,33.0,E wallet,Female,3.0,5,Laptop & Accessory,5,Single,7,0,16.0,1.0,2.0,2.0,191.96
5314,55315,0,21.0,Mobile Phone,2,33.0,Debit Card,Female,4.0,4,Others,3,Married,5,0,NaN,2.0,2.0,1.0,321.58
1213,51214,1,0.0,Mobile Phone,3,15.0,Credit Card,Male,2.0,3,Mobile Phone,1,Single,2,0,13.0,0.0,1.0,0.0,110.81
4087,54088,0,28.0,Mobile Phone,1,16.0,Debit Card,Female,4.0,4,Laptop & Accessory,3,Married,5,0,12.0,1.0,3.0,4.0,153.97
2839,52840,1,15.0,Computer,1,15.0,Debit Card,Male,3.0,6,Fashion,2,Single,4,0,16.0,1.0,2.0,4.0,231.50
1472,51473,1,NaN,Mobile Phone,3,28.0,E wallet,Male,2.0,3,Mobile Phone,3,Single,2,1,19.0,1.0,1.0,0.0,116.75
1329,51330,0,18.0,Phone,3,26.0,Debit Card,Female,2.0,3,Laptop & Accessory,3,Single,9,0,15.0,1.0,1.0,1.0,148.50
2920,52921,0,26.0,Mobile Phone,1,9.0,Debit Card,Female,3.0,5,Mobile Phone,2,Single,9,0,16.0,1.0,3.0,3.0,157.12
3186,53187,0,1.0,Mobile Phone,3,29.0,COD,Male,3.0,4,Mobile Phone,3,Single,3,0,12.0,2.0,2.0,3.0,159.47


In [29]:
# get information about the data
ecom_cus_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5630 entries, 0 to 5629
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   CustomerID                   5630 non-null   int64  
 1   Churn                        5630 non-null   int64  
 2   Tenure                       5366 non-null   float64
 3   PreferredLoginDevice         5630 non-null   object 
 4   CityTier                     5630 non-null   int64  
 5   WarehouseToHome              5379 non-null   float64
 6   PreferredPaymentMode         5630 non-null   object 
 7   Gender                       5630 non-null   object 
 8   HourSpendOnApp               5375 non-null   float64
 9   NumberOfDeviceRegistered     5630 non-null   int64  
 10  PreferedOrderCat             5630 non-null   object 
 11  SatisfactionScore            5630 non-null   int64  
 12  MaritalStatus                5630 non-null   object 
 13  NumberOfAddress   

#Data Preprocessing

There are no duplicates in the data for we will not be removing or deal with duplicates

In [30]:
# Checking for duplicates in the dataset
ecom_cus_df.duplicated().sum()

0

In [31]:
# drop columns that will not be used for modelling and analysis
ecom_cus_df.drop(columns=['CustomerID', 'NumberOfDeviceRegistered','NumberOfAddress'], inplace=True)

In [32]:
# print categorical, numerical and ordinal columns
categorical_columns = ecom_cus_df.select_dtypes(include=['object']).columns
numerical_columns = ecom_cus_df.select_dtypes(include=['float64']).columns
ordinal_columns = ecom_cus_df.select_dtypes(include=['int64']).columns

# print them
print(f"Categorical columns {categorical_columns}\n")
print(f"Numerical columns {numerical_columns}\n")
print(f"Ordinal columns {ordinal_columns}")

Categorical columns Index(['PreferredLoginDevice', 'PreferredPaymentMode', 'Gender',
       'PreferedOrderCat', 'MaritalStatus'],
      dtype='object')

Numerical columns Index(['Tenure', 'WarehouseToHome', 'HourSpendOnApp',
       'OrderAmountHikeFromlastYear', 'CouponUsed', 'OrderCount',
       'DaySinceLastOrder', 'CashbackAmount'],
      dtype='object')

Ordinal columns Index(['Churn', 'CityTier', 'SatisfactionScore', 'Complain'], dtype='object')


In [33]:
# check for missing values
ecom_cus_df.isna().sum()

,0
Churn,0
Tenure,264
PreferredLoginDevice,0
CityTier,0
WarehouseToHome,251
PreferredPaymentMode,0
Gender,0
HourSpendOnApp,255
PreferedOrderCat,0
SatisfactionScore,0


In [34]:
# lets check the descriptive statistics of the numerical columns before and after handling the missing valus
ecom_cus_df[numerical_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
Tenure,5366.0,10.189899,8.557241,0.0,2.00,9.00,16.0000,61.00
WarehouseToHome,5379.0,15.639896,8.531475,5.0,9.00,14.00,20.0000,127.00
HourSpendOnApp,5375.0,2.931535,0.721926,0.0,2.00,3.00,3.0000,5.00
OrderAmountHikeFromlastYear,5365.0,15.707922,3.675485,11.0,13.00,15.00,18.0000,26.00
CouponUsed,5374.0,1.751023,1.894621,0.0,1.00,1.00,2.0000,16.00
OrderCount,5372.0,3.008004,2.939680,1.0,1.00,2.00,3.0000,16.00
DaySinceLastOrder,5323.0,4.543491,3.654433,0.0,2.00,3.00,7.0000,46.00
CashbackAmount,5630.0,177.223030,49.207036,0.0,145.77,163.28,196.3925,324.99


For handling missing values, we will investigate potential strong relationships between customers who used coupons and other columns. We believe that a person's coupon usage could depend on various factors, such as their years spent on the platform. This analysis will help us decide whether to use a custom function or the median value to fill in the missing data, taking into consideration any outliers.

In [35]:
correlation_matrix = ecom_cus_df.select_dtypes(['float64','int64']).corr()  # This computes the Pearson correlation matrix
coupon_correlation = correlation_matrix['CouponUsed'].drop('CouponUsed')  # Extracts correlations with 'CouponUsed'
coupon_correlation

,CouponUsed
Churn,-0.008264
Tenure,0.129035
CityTier,0.021456
WarehouseToHome,-0.003935
HourSpendOnApp,0.191528
SatisfactionScore,0.017936
Complain,-0.008174
OrderAmountHikeFromlastYear,0.033201
OrderCount,0.745245
DaySinceLastOrder,0.358930


In [36]:
# print the mean for both Coupon used and OrderCount
print(ecom_cus_df['CouponUsed'].mean())
print(ecom_cus_df['OrderCount'].mean())

1.7510234462225531
3.0080044676098288


Based on the correlations, it's clear that the use of coupons is closely related to the number of orders placed by each person. On average, we found that the most common order counts associated with coupon use are 2 and 3, respectively.

To handle missing values, we've implemented a function that checks for missing data points. If a data point is missing and the associated order count is greater than 3, the function assigns a coupon value of 3 (a predetermined number); otherwise, it assigns a value of 1.

For any remaining missing values in other columns, we'll use the median imputation method.

In [37]:
# create the function for the imputation
def fill_coupon_based_on_order(row):
  if pd.isna(row['CouponUsed']):
    if row['OrderCount'] > 3:
      return 3 #Since mean is 2, if order count is greater than mean order count, return a value greater than mean coupon used.
    else:
      return 1 #Else return a value less than the mean coupon used.
  else:
    return row['CouponUsed']

ecom_cus_df['CouponUsed'] = ecom_cus_df.apply(fill_coupon_based_on_order, axis=1)

In [40]:
# impute the missing values for the remaining columns
medians = ecom_cus_df[numerical_columns].median()
ecom_cus_df[numerical_columns] = ecom_cus_df[numerical_columns].fillna(medians)

In [41]:
# now check the descriptive statistics again
ecom_cus_df[numerical_columns].describe().T

,count,mean,std,min,25%,50%,75%,max
Tenure,5630.0,10.134103,8.357951,0.0,3.00,9.00,15.0000,61.00
WarehouseToHome,5630.0,15.566785,8.345961,5.0,9.00,14.00,20.0000,127.00
HourSpendOnApp,5630.0,2.934636,0.705528,0.0,2.00,3.00,3.0000,5.00
OrderAmountHikeFromlastYear,5630.0,15.674600,3.591058,11.0,13.00,15.00,18.0000,26.00
CouponUsed,5630.0,1.758437,1.863501,0.0,1.00,1.00,2.0000,16.00
OrderCount,5630.0,2.961812,2.879248,1.0,1.00,2.00,3.0000,16.00
DaySinceLastOrder,5630.0,4.459325,3.570626,0.0,2.00,3.00,7.0000,46.00
CashbackAmount,5630.0,177.223030,49.207036,0.0,145.77,163.28,196.3925,324.99


In [45]:
print(f"{ecom_cus_df['PreferredLoginDevice'].value_counts()}\n")
print(f"{ecom_cus_df['PreferredPaymentMode'].value_counts()}")

PreferredLoginDevice
Mobile Phone    2765
Computer        1634
Phone           1231
Name: count, dtype: int64

PreferredPaymentMode
Debit Card          2314
Credit Card         1501
E wallet             614
UPI                  414
COD                  365
CC                   273
Cash on Delivery     149
Name: count, dtype: int64


In [46]:
# Replacing some values
# Replace values in multiple columns
ecom_cus_df['PreferredLoginDevice'] = ecom_cus_df['PreferredLoginDevice'].replace(['Phone', 'Mobile Phone'], 'Mobile')

ecom_cus_df['PreferredPaymentMode'] = ecom_cus_df['PreferredPaymentMode'].replace({
    'COD': 'Cash on Delivery',
    'CC': 'Credit Card'
})

# Exloratory Data Analysis